# English Football Pyramid - Análisis Completo (1993-2025)

Análisis integral de las 5 divisiones del fútbol inglés: Premier League, Championship, League One, League Two y National League.

## 1. Configuración e Importación de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Librerías importadas correctamente")

In [ ]:
# Cargar datasets
df = pd.read_csv('english_leagues_completo.csv')
tracking = pd.read_csv('english_leagues_tracking.csv')

print(f"📊 Dataset Principal Cargado")
print(f"   Total registros: {len(df):,}")
print(f"   Divisiones: {df['Division'].nunique()}")
print(f"   Temporadas: {df['Temporada'].nunique()}")
print(f"   Equipos únicos: {df['Equipo'].nunique()}")
print(f"\n📈 Tracking Cargado")
print(f"   Equipos rastreados: {len(tracking):,}")

# Mostrar primeras filas
df.head(10)

## 2. Resumen General del Dataset

In [ ]:
# Resumen por división
resumen = df.groupby('Division').agg({
    'Temporada': 'nunique',
    'Equipo': 'nunique',
    'PJ': 'sum',
    'GF': 'sum',
    'GC': 'sum',
    'Pts': 'sum'
}).reset_index()

resumen.columns = ['División', 'Temporadas', 'Equipos', 'Partidos', 'Goles_Favor', 'Goles_Contra', 'Puntos_Total']
resumen['Registros'] = df.groupby('Division').size().values
resumen['Goles_Promedio_Partido'] = (resumen['Goles_Favor'] + resumen['Goles_Contra']) / resumen['Partidos']

# Ordenar por nivel de división
division_order = ['Premier League', 'Championship', 'League One', 'League Two', 'National League']
resumen['Order'] = resumen['División'].map({div: i for i, div in enumerate(division_order)})
resumen = resumen.sort_values('Order').drop('Order', axis=1)

print("="*90)
print("RESUMEN GENERAL POR DIVISIÓN")
print("="*90)
print(resumen.to_string(index=False))
print("\n" + "="*90)

In [ ]:
# Visualización: Distribución de equipos por división
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Equipos únicos por división
equipos_div = df.groupby('Division')['Equipo'].nunique().reindex(division_order)
axes[0].barh(division_order, equipos_div.values, color=['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6'])
axes[0].set_xlabel('Número de Equipos Únicos', fontsize=12, fontweight='bold')
axes[0].set_title('Equipos Únicos por División', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
for i, v in enumerate(equipos_div.values):
    axes[0].text(v + 1, i, str(v), va='center', fontweight='bold')

# Registros por división
registros_div = df.groupby('Division').size().reindex(division_order)
axes[1].barh(division_order, registros_div.values, color=['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6'])
axes[1].set_xlabel('Número de Registros', fontsize=12, fontweight='bold')
axes[1].set_title('Registros Totales por División', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
for i, v in enumerate(registros_div.values):
    axes[1].text(v + 5, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Análisis por División

### 3.1 Premier League (Nivel 1)

In [ ]:
premier = df[df['Division'] == 'Premier League'].copy()

print("="*70)
print("PREMIER LEAGUE - ANÁLISIS")
print("="*70)
print(f"Temporadas: {premier['Temporada'].nunique()}")
print(f"Equipos únicos: {premier['Equipo'].nunique()}")
print(f"Total registros: {len(premier):,}")
print(f"Periodo: {premier['Temporada'].min()} - {premier['Temporada'].max()}")

# Campeones históricos
print("\n" + "="*70)
print("CAMPEONES HISTÓRICOS")
print("="*70)
campeones = premier[premier['Pos'] == 1].groupby('Equipo').size().sort_values(ascending=False)
for equipo, titulos in campeones.items():
    print(f"{equipo}: {titulos} {'título' if titulos == 1 else 'títulos'}")

# Equipos en todas las temporadas
equipos_siempre = premier.groupby('Equipo')['Temporada'].nunique()
todas_temporadas = equipos_siempre[equipos_siempre == premier['Temporada'].nunique()]
print(f"\n" + "="*70)
print(f"EQUIPOS EN TODAS LAS {premier['Temporada'].nunique()} TEMPORADAS")
print("="*70)
for equipo in todas_temporadas.index:
    stats = premier[premier['Equipo'] == equipo]
    print(f"{equipo:15} - Mejor: {stats['Pos'].min()}°, Peor: {stats['Pos'].max()}°")

# Récord de puntos
record = premier.nlargest(1, 'Pts').iloc[0]
print(f"\n" + "="*70)
print("RÉCORD DE PUNTOS")
print("="*70)
print(f"{record['Equipo']} - {record['Pts']} pts ({record['Temporada']})")
print(f"Detalle: {record['G']}G {record['E']}E {record['P']}P | GF:{record['GF']} GC:{record['GC']}")

In [ ]:
# Visualización Premier League: Evolución de puntos del campeón
campeones_pl = premier[premier['Pos'] == 1].sort_values('Temporada')

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Puntos del campeón por temporada
axes[0].plot(range(len(campeones_pl)), campeones_pl['Pts'].values, marker='o', linewidth=2, markersize=6, color='#3498db')
axes[0].axhline(campeones_pl['Pts'].mean(), color='red', linestyle='--', label=f'Promedio: {campeones_pl["Pts"].mean():.1f} pts')
axes[0].set_ylabel('Puntos', fontsize=12, fontweight='bold')
axes[0].set_title('Premier League - Puntos del Campeón por Temporada', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()
axes[0].set_xticks(range(len(campeones_pl)))
axes[0].set_xticklabels(campeones_pl['Temporada'].values, rotation=45, ha='right')

# Récords: Top 10 mejores temporadas
top10_pl = premier.nlargest(10, 'Pts')
labels = [f"{row['Equipo']}\n{row['Temporada']}" for _, row in top10_pl.iterrows()]
axes[1].barh(range(10), top10_pl['Pts'].values, color='#2ecc71')
axes[1].set_yticks(range(10))
axes[1].set_yticklabels(labels)
axes[1].set_xlabel('Puntos', fontsize=12, fontweight='bold')
axes[1].set_title('Premier League - Top 10 Mejores Temporadas por Puntos', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
for i, v in enumerate(top10_pl['Pts'].values):
    axes[1].text(v + 1, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 3.2 Championship (Nivel 2)

In [ ]:
championship = df[df['Division'] == 'Championship'].copy()

print("="*70)
print("CHAMPIONSHIP - ANÁLISIS")
print("="*70)
print(f"Temporadas: {championship['Temporada'].nunique()}")
print(f"Equipos únicos: {championship['Equipo'].nunique()}")
print(f"Total registros: {len(championship):,}")
print(f"Periodo: {championship['Temporada'].min()} - {championship['Temporada'].max()}")

# Campeones más frecuentes
print("\n" + "="*70)
print("CAMPEONES MÁS FRECUENTES")
print("="*70)
campeones_champ = championship[championship['Pos'] == 1].groupby('Equipo').size().sort_values(ascending=False).head(10)
for equipo, titulos in campeones_champ.items():
    print(f"{equipo}: {titulos} {'título' if titulos == 1 else 'títulos'}")

# Récord de puntos
record_champ = championship.nlargest(1, 'Pts').iloc[0]
print(f"\n" + "="*70)
print("RÉCORD DE PUNTOS")
print("="*70)
print(f"{record_champ['Equipo']} - {record_champ['Pts']} pts ({record_champ['Temporada']})")
print(f"Detalle: {record_champ['G']}G {record_champ['E']}E {record_champ['P']}P | GF:{record_champ['GF']} GC:{record_champ['GC']}")

# Equipos con más temporadas en Championship
print(f"\n" + "="*70)
print("EQUIPOS CON MÁS TEMPORADAS EN CHAMPIONSHIP")
print("="*70)
temp_champ = championship.groupby('Equipo')['Temporada'].nunique().sort_values(ascending=False).head(10)
for equipo, temporadas in temp_champ.items():
    print(f"{equipo}: {temporadas} temporadas")

In [ ]:
# Visualización Championship: Distribución de puntos y goles
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribución de puntos
axes[0].hist(championship['Pts'], bins=30, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[0].axvline(championship['Pts'].mean(), color='red', linestyle='--', linewidth=2, label=f'Promedio: {championship["Pts"].mean():.1f}')
axes[0].set_xlabel('Puntos', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[0].set_title('Championship - Distribución de Puntos', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Relación Goles a Favor vs Puntos
axes[1].scatter(championship['GF'], championship['Pts'], alpha=0.5, s=50, color='#2ecc71')
axes[1].set_xlabel('Goles a Favor', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Puntos', fontsize=12, fontweight='bold')
axes[1].set_title('Championship - Goles a Favor vs Puntos', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

# Línea de tendencia
z = np.polyfit(championship['GF'], championship['Pts'], 1)
p = np.poly1d(z)
axes[1].plot(championship['GF'], p(championship['GF']), "r--", alpha=0.8, linewidth=2, label=f'Tendencia: y={z[0]:.2f}x+{z[1]:.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()

### 3.3 League One (Nivel 3)

In [ ]:
league_one = df[df['Division'] == 'League One'].copy()

print("="*70)
print("LEAGUE ONE - ANÁLISIS")
print("="*70)
print(f"Temporadas: {league_one['Temporada'].nunique()}")
print(f"Equipos únicos: {league_one['Equipo'].nunique()}")
print(f"Total registros: {len(league_one):,}")
print(f"Periodo: {league_one['Temporada'].min()} - {league_one['Temporada'].max()}")

# Campeones más frecuentes
print("\n" + "="*70)
print("CAMPEONES MÁS FRECUENTES")
print("="*70)
campeones_l1 = league_one[league_one['Pos'] == 1].groupby('Equipo').size().sort_values(ascending=False).head(10)
for equipo, titulos in campeones_l1.items():
    print(f"{equipo}: {titulos} {'título' if titulos == 1 else 'títulos'}")

# Récord de puntos
record_l1 = league_one.nlargest(1, 'Pts').iloc[0]
print(f"\n" + "="*70)
print("RÉCORD DE PUNTOS")
print("="*70)
print(f"{record_l1['Equipo']} - {record_l1['Pts']} pts ({record_l1['Temporada']})")
print(f"Detalle: {record_l1['G']}G {record_l1['E']}E {record_l1['P']}P | GF:{record_l1['GF']} GC:{record_l1['GC']}")

# Promedio de goles por partido
goles_partido_l1 = (league_one['GF'].sum() + league_one['GC'].sum()) / league_one['PJ'].sum()
print(f"\nPromedio de goles por partido: {goles_partido_l1:.2f}")

### 3.4 League Two (Nivel 4)

In [ ]:
league_two = df[df['Division'] == 'League Two'].copy()

print("="*70)
print("LEAGUE TWO - ANÁLISIS")
print("="*70)
print(f"Temporadas: {league_two['Temporada'].nunique()}")
print(f"Equipos únicos: {league_two['Equipo'].nunique()}")
print(f"Total registros: {len(league_two):,}")
print(f"Periodo: {league_two['Temporada'].min()} - {league_two['Temporada'].max()}")

# Campeones más frecuentes
print("\n" + "="*70)
print("CAMPEONES MÁS FRECUENTES")
print("="*70)
campeones_l2 = league_two[league_two['Pos'] == 1].groupby('Equipo').size().sort_values(ascending=False).head(10)
for equipo, titulos in campeones_l2.items():
    print(f"{equipo}: {titulos} {'título' if titulos == 1 else 'títulos'}")

# Récord de puntos
record_l2 = league_two.nlargest(1, 'Pts').iloc[0]
print(f"\n" + "="*70)
print("RÉCORD DE PUNTOS")
print("="*70)
print(f"{record_l2['Equipo']} - {record_l2['Pts']} pts ({record_l2['Temporada']})")
print(f"Detalle: {record_l2['G']}G {record_l2['E']}E {record_l2['P']}P | GF:{record_l2['GF']} GC:{record_l2['GC']}")

### 3.5 National League (Nivel 5)

In [ ]:
national = df[df['Division'] == 'National League'].copy()

print("="*70)
print("NATIONAL LEAGUE - ANÁLISIS")
print("="*70)
print(f"Temporadas: {national['Temporada'].nunique()}")
print(f"Equipos únicos: {national['Equipo'].nunique()}")
print(f"Total registros: {len(national):,}")
print(f"Periodo: {national['Temporada'].min()} - {national['Temporada'].max()}")

# Campeones más frecuentes
print("\n" + "="*70)
print("CAMPEONES MÁS FRECUENTES")
print("="*70)
campeones_nl = national[national['Pos'] == 1].groupby('Equipo').size().sort_values(ascending=False).head(10)
for equipo, titulos in campeones_nl.items():
    print(f"{equipo}: {titulos} {'título' if titulos == 1 else 'títulos'}")

# Récord de puntos
record_nl = national.nlargest(1, 'Pts').iloc[0]
print(f"\n" + "="*70)
print("RÉCORD DE PUNTOS")
print("="*70)
print(f"{record_nl['Equipo']} - {record_nl['Pts']} pts ({record_nl['Temporada']})")
print(f"Detalle: {record_nl['G']}G {record_nl['E']}E {record_nl['P']}P | GF:{record_nl['GF']} GC:{record_nl['GC']}")

## 4. Comparación Entre Divisiones

In [ ]:
# Comparación de promedios por división
comparacion = df.groupby('Division').agg({
    'Pts': 'mean',
    'GF': 'mean',
    'GC': 'mean',
    'G': 'mean',
    'E': 'mean',
    'P': 'mean'
}).round(2)

comparacion.columns = ['Pts_Prom', 'GF_Prom', 'GC_Prom', 'Victorias_Prom', 'Empates_Prom', 'Derrotas_Prom']
comparacion = comparacion.reindex(division_order)

print("="*90)
print("COMPARACIÓN DE PROMEDIOS POR DIVISIÓN")
print("="*90)
print(comparacion)
print("\n" + "="*90)

In [ ]:
# Visualización: Box plots comparativos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Puntos
df_ordered = df.copy()
df_ordered['Division'] = pd.Categorical(df_ordered['Division'], categories=division_order, ordered=True)
df_ordered = df_ordered.sort_values('Division')

sns.boxplot(data=df_ordered, x='Division', y='Pts', ax=axes[0, 0], palette='husl')
axes[0, 0].set_title('Distribución de Puntos por División', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# Goles a Favor
sns.boxplot(data=df_ordered, x='Division', y='GF', ax=axes[0, 1], palette='husl')
axes[0, 1].set_title('Distribución de Goles a Favor por División', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# Victorias
sns.boxplot(data=df_ordered, x='Division', y='G', ax=axes[1, 0], palette='husl')
axes[1, 0].set_title('Distribución de Victorias por División', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Diferencia de goles (convertir a numérico)
df_ordered['Dif_num'] = df_ordered['Dif'].astype(str).str.replace('+', '').astype(int)
sns.boxplot(data=df_ordered, x='Division', y='Dif_num', ax=axes[1, 1], palette='husl')
axes[1, 1].set_title('Distribución de Diferencia de Goles por División', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('')
axes[1, 1].set_ylabel('Diferencia de Goles')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)
axes[1, 1].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 5. Análisis de Movilidad Entre Divisiones

In [ ]:
# Equipos con mayor movilidad
if 'Divisiones_Jugadas' in tracking.columns:
    movilidad = tracking[['Equipo', 'Total_Temporadas', 'Divisiones_Jugadas', 'Mejor_Division']].copy()
    movilidad = movilidad.sort_values('Divisiones_Jugadas', ascending=False)
    
    print("="*70)
    print("TOP 20 EQUIPOS CON MAYOR MOVILIDAD ENTRE DIVISIONES")
    print("="*70)
    print(movilidad.head(20).to_string(index=False))
    
    # Equipos que han jugado en todas las divisiones
    todas_divisiones = movilidad[movilidad['Divisiones_Jugadas'] == 5]
    if len(todas_divisiones) > 0:
        print(f"\n" + "="*70)
        print(f"EQUIPOS QUE HAN JUGADO EN LAS 5 DIVISIONES ({len(todas_divisiones)})")
        print("="*70)
        for _, row in todas_divisiones.iterrows():
            print(f"{row['Equipo']:20} - {int(row['Total_Temporadas'])} temporadas")
else:
    print("Columna 'Divisiones_Jugadas' no encontrada en tracking")

In [ ]:
# Visualización de movilidad
if 'Divisiones_Jugadas' in tracking.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribución de equipos por número de divisiones jugadas
    div_jugadas = tracking['Divisiones_Jugadas'].value_counts().sort_index()
    axes[0].bar(div_jugadas.index, div_jugadas.values, color='#9b59b6', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Número de Divisiones Jugadas', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Número de Equipos', fontsize=12, fontweight='bold')
    axes[0].set_title('Distribución de Equipos por Movilidad', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(div_jugadas.values):
        axes[0].text(div_jugadas.index[i], v + 1, str(v), ha='center', fontweight='bold')
    
    # Top 15 equipos más viajados
    top_movilidad = movilidad.nlargest(15, 'Divisiones_Jugadas')
    axes[1].barh(range(15), top_movilidad['Divisiones_Jugadas'].values, color='#e74c3c')
    axes[1].set_yticks(range(15))
    axes[1].set_yticklabels(top_movilidad['Equipo'].values)
    axes[1].set_xlabel('Divisiones Jugadas', fontsize=12, fontweight='bold')
    axes[1].set_title('Top 15 Equipos con Mayor Movilidad', fontsize=14, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)
    for i, v in enumerate(top_movilidad['Divisiones_Jugadas'].values):
        axes[1].text(v + 0.1, i, str(int(v)), va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 6. Funciones de Consulta Personalizadas

In [ ]:
def ver_historial_equipo(equipo_nombre):
    """
    Muestra el historial completo de un equipo a través de todas las divisiones
    """
    # Buscar coincidencias
    equipos_disponibles = df['Equipo'].unique()
    coincidencias = [e for e in equipos_disponibles if equipo_nombre.lower() in e.lower()]
    
    if not coincidencias:
        print(f"❌ No se encontró ningún equipo con '{equipo_nombre}'")
        return
    
    if len(coincidencias) > 1:
        print(f"⚠️  Se encontraron {len(coincidencias)} equipos:")
        for e in coincidencias:
            print(f"   - {e}")
        print("\nPor favor, especifica mejor el nombre del equipo")
        return
    
    equipo = coincidencias[0]
    historial = df[df['Equipo'] == equipo].sort_values('Temporada')
    
    print("\n" + "="*80)
    print(f"HISTORIAL COMPLETO: {equipo}")
    print("="*80)
    print(f"Temporadas jugadas: {len(historial)}")
    print(f"Divisiones jugadas: {historial['Division'].nunique()}")
    print(f"\nPor división:")
    for div in division_order:
        div_data = historial[historial['Division'] == div]
        if len(div_data) > 0:
            print(f"  {div}: {len(div_data)} temporadas")
    
    print(f"\nMejor posición: {historial['Pos'].min()}° ({historial[historial['Pos'] == historial['Pos'].min()]['Temporada'].values[0]} - {historial[historial['Pos'] == historial['Pos'].min()]['Division'].values[0]})")
    print(f"Peor posición: {historial['Pos'].max()}° ({historial[historial['Pos'] == historial['Pos'].max()]['Temporada'].values[0]} - {historial[historial['Pos'] == historial['Pos'].max()]['Division'].values[0]})")
    print(f"\nTítulos (1° lugar): {len(historial[historial['Pos'] == 1])}")
    
    print("\n" + "="*80)
    print("HISTORIAL DETALLADO")
    print("="*80)
    print(historial[['Temporada', 'Division', 'Pos', 'PJ', 'G', 'E', 'P', 'Pts', 'GF', 'GC']].to_string(index=False))
    
    return historial

# Ejemplo de uso:
# ver_historial_equipo('Leicester')

In [ ]:
def ver_temporada_division(temporada, division):
    """
    Muestra la clasificación completa de una división en una temporada específica
    """
    temp_data = df[(df['Temporada'] == temporada) & (df['Division'] == division)].sort_values('Pos')
    
    if len(temp_data) == 0:
        print(f"❌ No se encontraron datos para {division} en temporada {temporada}")
        return
    
    print("\n" + "="*90)
    print(f"{division.upper()} - TEMPORADA {temporada}")
    print("="*90)
    print(temp_data[['Pos', 'Equipo', 'PJ', 'G', 'E', 'P', 'Pts', 'GF', 'GC', 'Dif']].to_string(index=False))
    print("\n" + "="*90)
    
    # Estadísticas de la temporada
    print(f"\nEstadísticas de la temporada:")
    print(f"  Campeón: {temp_data.iloc[0]['Equipo']} ({temp_data.iloc[0]['Pts']} pts)")
    print(f"  Promedio de puntos: {temp_data['Pts'].mean():.2f}")
    print(f"  Total de goles: {temp_data['GF'].sum()}")
    print(f"  Goles por partido: {(temp_data['GF'].sum() + temp_data['GC'].sum()) / temp_data['PJ'].sum():.2f}")
    
    return temp_data

# Ejemplo de uso:
# ver_temporada_division('2015-16', 'Premier League')  # Temporada del Leicester

In [ ]:
def comparar_equipos(*equipos):
    """
    Compara el historial de múltiples equipos a través de todas las divisiones
    """
    comparacion = []
    
    for equipo_nombre in equipos:
        coincidencias = [e for e in df['Equipo'].unique() if equipo_nombre.lower() in e.lower()]
        
        if len(coincidencias) == 1:
            equipo = coincidencias[0]
            historial = df[df['Equipo'] == equipo]
            
            comparacion.append({
                'Equipo': equipo,
                'Temporadas': len(historial),
                'Divisiones': historial['Division'].nunique(),
                'Títulos': len(historial[historial['Pos'] == 1]),
                'Top 3': len(historial[historial['Pos'] <= 3]),
                'Mejor_Pos': historial['Pos'].min(),
                'Peor_Pos': historial['Pos'].max(),
                'Total_Pts': historial['Pts'].sum(),
                'Prom_Pts': historial['Pts'].mean(),
                'Total_GF': historial['GF'].sum(),
                'Total_GC': historial['GC'].sum()
            })
        elif len(coincidencias) == 0:
            print(f"⚠️  No se encontró '{equipo_nombre}'")
        else:
            print(f"⚠️  Múltiples coincidencias para '{equipo_nombre}': {coincidencias}")
    
    if comparacion:
        comp_df = pd.DataFrame(comparacion)
        print("\n" + "="*120)
        print("COMPARACIÓN DE EQUIPOS")
        print("="*120)
        print(comp_df.to_string(index=False))
        print("\n" + "="*120)
        return comp_df
    
    return None

# Ejemplo de uso:
# comparar_equipos('Leicester', 'Southampton', 'Newcastle')

## 7. Récords Históricos por División

In [ ]:
print("="*90)
print("RÉCORDS HISTÓRICOS POR DIVISIÓN")
print("="*90)

for division in division_order:
    div_data = df[df['Division'] == division]
    
    print(f"\n{'='*90}")
    print(f"{division.upper()}")
    print(f"{'='*90}")
    
    # Más puntos
    record_pts = div_data.nlargest(1, 'Pts').iloc[0]
    print(f"MÁS PUNTOS: {record_pts['Equipo']} - {record_pts['Pts']} pts ({record_pts['Temporada']})")
    
    # Más goles a favor
    record_gf = div_data.nlargest(1, 'GF').iloc[0]
    print(f"MÁS GOLES A FAVOR: {record_gf['Equipo']} - {record_gf['GF']} goles ({record_gf['Temporada']})")
    
    # Menos goles en contra
    record_gc = div_data.nsmallest(1, 'GC').iloc[0]
    print(f"MENOS GOLES EN CONTRA: {record_gc['Equipo']} - {record_gc['GC']} goles ({record_gc['Temporada']})")
    
    # Más victorias
    record_w = div_data.nlargest(1, 'G').iloc[0]
    print(f"MÁS VICTORIAS: {record_w['Equipo']} - {record_w['G']} victorias ({record_w['Temporada']})")
    
    # Mejor diferencia de goles
    div_data_copy = div_data.copy()
    div_data_copy['Dif_num'] = div_data_copy['Dif'].astype(str).str.replace('+', '').astype(int)
    record_dif = div_data_copy.nlargest(1, 'Dif_num').iloc[0]
    print(f"MEJOR DIFERENCIA: {record_dif['Equipo']} - {record_dif['Dif']} ({record_dif['Temporada']})")

print("\n" + "="*90)

## 8. Análisis de Tendencias Temporales

In [ ]:
# Evolución de goles promedio por temporada
goles_temporada = df.groupby('Temporada').apply(
    lambda x: (x['GF'].sum() + x['GC'].sum()) / x['PJ'].sum()
)

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(range(len(goles_temporada)), goles_temporada.values, marker='o', linewidth=2, markersize=5)
ax.axhline(goles_temporada.mean(), color='red', linestyle='--', label=f'Promedio: {goles_temporada.mean():.2f}')
ax.set_xlabel('Temporada', fontsize=12, fontweight='bold')
ax.set_ylabel('Goles por Partido', fontsize=12, fontweight='bold')
ax.set_title('Evolución de Goles Promedio por Partido (Todas las Divisiones)', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
ax.legend()
ax.set_xticks(range(0, len(goles_temporada), 3))
ax.set_xticklabels(goles_temporada.index[::3], rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 9. Consultas Personalizadas

Usa las funciones definidas arriba para hacer tus propias consultas:

In [ ]:
# Ejemplo 1: Ver historial de un equipo
ver_historial_equipo('Leicester')

In [ ]:
# Ejemplo 2: Ver temporada específica
ver_temporada_division('2015-16', 'Premier League')  # Temporada del Leicester campeón

In [ ]:
# Ejemplo 3: Comparar equipos
comparar_equipos('Man City', 'Liverpool', 'Chelsea', 'Arsenal')

In [ ]:
# Ejemplo 4: Equipos que más han ascendido/descendido
# Analizar equipos en Championship que llegaron a Premier League
equipos_championship = df[df['Division'] == 'Championship']['Equipo'].unique()
equipos_premier = df[df['Division'] == 'Premier League']['Equipo'].unique()
equipos_ascendidos = set(equipos_championship) & set(equipos_premier)

print(f"\nEquipos que han jugado en Championship y Premier League: {len(equipos_ascendidos)}")
print("\nAlgunos ejemplos:")
for equipo in list(equipos_ascendidos)[:10]:
    print(f"  - {equipo}")

## 10. Exportar Resultados

Guarda análisis específicos para uso posterior

In [ ]:
# Crear resumen ejecutivo
resumen_ejecutivo = {
    'Total_Registros': len(df),
    'Total_Divisiones': df['Division'].nunique(),
    'Total_Temporadas': df['Temporada'].nunique(),
    'Total_Equipos': df['Equipo'].nunique(),
    'Total_Partidos': df['PJ'].sum(),
    'Total_Goles': df['GF'].sum() + df['GC'].sum(),
    'Goles_Por_Partido_Promedio': (df['GF'].sum() + df['GC'].sum()) / df['PJ'].sum()
}

print("\n" + "="*70)
print("RESUMEN EJECUTIVO - ENGLISH FOOTBALL PYRAMID")
print("="*70)
for key, value in resumen_ejecutivo.items():
    if isinstance(value, float):
        print(f"{key.replace('_', ' ')}: {value:.2f}")
    else:
        print(f"{key.replace('_', ' ')}: {value:,}")
print("="*70)

# Opcional: Guardar en CSV
# pd.DataFrame([resumen_ejecutivo]).to_csv('resumen_ejecutivo_english_leagues.csv', index=False)
# print("\n✓ Resumen guardado en 'resumen_ejecutivo_english_leagues.csv'")

---

## 📝 Notas Finales

Este notebook proporciona un análisis completo de las 5 divisiones del fútbol inglés. Puedes:

- Ejecutar todas las celdas para obtener el análisis completo
- Usar las funciones personalizadas para consultas específicas
- Modificar las visualizaciones según tus necesidades
- Exportar resultados para uso posterior

**Dataset**: `english_leagues_completo.csv` (2,516 registros)
**Tracking**: `english_leagues_tracking.csv` (159 equipos)
**Periodo**: 1993-2025 (32 temporadas)

---

🤖 Notebook generado como parte del proyecto English Football Pyramid v3.0